https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.

https://github.com/bomri/SlowFast/blob/master/slowfast/datasets/loader.py

https://github.com/bomri/SlowFast/blob/master/slowfast/datasets/ava_dataset.py

https://github.com/HHTseng/video-classification/blob/master/ResNetCRNN_varylength/UCF101_ResNetCRNN_varlen.py

In [ ]:
import os
#import av
import cv2
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

class BasicVideoDataset(Dataset):
    def __init__(self, annotations_file, video_dir, IMG_SIZE):
        self.video_labels = pd.read_csv(annotations_file, sep='\t')
        self.video_dir = video_dir
        self.IMG_SIZE = IMG_SIZE

    def __len__(self):
        return len(self.video_labels)

    def crop_center_square(self, frame):
        y, x = frame.shape[0:2]
        min_dim = min(y, x)
        start_x = (x // 2) - (min_dim // 2)
        start_y = (y // 2) - (min_dim // 2)
        return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


    def load_video(self, path, begin, end, max_frames=0, resize=(10, 10)):
        cap = cv2.VideoCapture(path)
        frames = []

        frame_index=begin
        try:
            while True and frame_index <= end:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
                ret, frame = cap.read()
                if not ret:
                    break
                frame = self.crop_center_square(frame)
                frame = cv2.resize(frame, resize)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
                frames.append(frame)
                frame_index+=1

                if len(frames) == max_frames:
                    break
        finally:
            cap.release()
        return torch.from_numpy(np.array(frames))

    def __getitem__(self, idx):
        filename  = os.path.join(self.video_dir, self.video_labels.iloc[idx]['attachment_id']+".mp4")
        label = self.video_labels.iloc[idx]['text']
        begin = self.video_labels.iloc[idx]['begin']
        end = self.video_labels.iloc[idx]['end']
        frames = self.load_video(filename, begin, end, resize=(self.IMG_SIZE, self.IMG_SIZE)) # Загрузка видео!!!!
        return frames, label

* The __init__ function is run once when instantiating the Dataset object. We initialize the directory containing the images, the annotations file, and both transforms (covered in more detail in the next section).
* The __len__ function returns the number of samples in our dataset.
* The __getitem__ function loads and returns a sample from the dataset at the given index idx.

In [ ]:
annotations_file = "/content/SLOVO_DF_SHORT.tsv"
video_dir = "/content/video"
IMG_SIZE = 224

In [ ]:
training_data = BasicVideoDataset(annotations_file, video_dir, IMG_SIZE=IMG_SIZE)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True)

In [ ]:
train_frames, train_labels = next(iter(train_dataloader))

In [ ]:
from PIL import Image
import imageio
import numpy as np
from tensorflow_docs.vis import embed

MAX_SEQ_LENGTH = 300


def to_gif(images, i, label):
  new_images = list()
  for image in images[:MAX_SEQ_LENGTH]:
      new_images.append(image.numpy())
  imageio.mimsave(f"{label}_{i}.gif", new_images, duration=100)

gifs = list()

data_loader_iterator = iter(train_dataloader)
for i in range(3):
  train_frames, train_labels = next(data_loader_iterator)
  to_gif( train_frames[0], i, train_labels[0])
  gifs.append(f"{train_labels[0]}_{i}.gif")


In [ ]:
print(gifs[0])
embed.embed_file(gifs[0])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(gifs[1])
embed.embed_file(gifs[1])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(gifs[2])
embed.embed_file(gifs[2])

Output hidden; open in https://colab.research.google.com to view.

In [11]:
from google.colab import files
for gif in gifs:
  files.download(gif)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Новый раздел